In [2]:
#!pip install torch transformers nltk

# import nessaries libraries:
import nltk
nltk.download('punkt')

import os
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import math
import re
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Set the device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

[nltk_data] Downloading package punkt to C:\Users\Pete
[nltk_data]     P\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using device: cpu


## Data Preprocessing

In [3]:
# Load and Parse Data

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

# Set the pad_token to the eos_token and specify padding side
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'  # For decoder-only models like GPT-2

# Define the paths to the dataset files
lines_file = "dataset/movie_lines.txt"
conversations_file = "dataset/movie_conversations.txt"

# Function to load and parse the movie lines
def load_lines(file_path):
    """
    Load and parse the movie lines from the dataset.
    Returns a dictionary mapping line IDs to text.
    """
    lines = {}
    # Open the file with the appropriate encoding to handle special characters
    with open(file_path, encoding='iso-8859-1') as f:
        for line in f:
            # Split each line into its components
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 5:
                line_id = parts[0]  # Unique identifier for the line
                text = parts[4]     # The actual dialogue text
                lines[line_id] = text  # Store in a dictionary
    return lines

# Function to load and parse the movie conversations
def load_conversations(file_path):
    """
    Load and parse the movie conversations from the dataset.
    Returns a list of conversations, each conversation is a list of line IDs.
    """
    conversations = []
    # Open the file with the appropriate encoding
    with open(file_path, encoding='iso-8859-1') as f:
        for line in f:
            # Split each line into its components
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 4:
                # The fourth part contains the list of line IDs in a conversation
                line_ids = eval(parts[3])  # Convert the string representation of the list to an actual list
                conversations.append(line_ids)
    return conversations

# Function to extract dialogues based on the conversations and lines
def extract_dialogues(conversations, lines):
    """
    Extract the dialogues from the conversations using the line IDs.
    Returns a list of dialogues, each dialogue is a list of utterances.
    """
    dialogues = []
    for conv in conversations:
        conv_dialogues = []
        for line_id in conv:
            if line_id in lines:
                conv_dialogues.append(lines[line_id])  # Append the dialogue text for each line ID
        dialogues.append(conv_dialogues)
    return dialogues

# Load the data
lines = load_lines(lines_file)
conversations = load_conversations(conversations_file)
dialogues = extract_dialogues(conversations, lines)

# Display sample dialogues for inspection
print("Sample Dialogues:")
for i in range(2):
    print(f"Conversation {i+1}: {dialogues[i]}")


C:\Users\Pete P\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Sample Dialogues:
Conversation 1: ['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.', "Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part.  Please.', "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"]
Conversation 2: ["You're asking me out.  That's so cute. What's your name again?", 'Forget it.']


#### Explanation:
- The pad_token is set to the eos_token (<|endoftext|>) to handle padding.
- The padding side is set to 'left' because GPT-2 is a decoder-only model.
- Load the dataset files and parse them to extract dialogues.
- The load_lines function reads movie_lines.txt and stores each line's text with its unique ID.
- The load_conversations function reads movie_conversations.txt and extracts sequences of line IDs representing conversations.
- The extract_dialogues function links line IDs to their actual text to reconstruct the conversations.

### Prepare Data for Training

In [4]:
# Set the maximum number of previous exchanges to use as context
max_history = 5

# Function to build input-target pairs from dialogues
def build_inputs_targets(dialogues):
    """
    Build input-target pairs for training.
    Each input consists of a context (history of utterances),
    and the target is the next reply.
    """
    inputs = []
    for conv in dialogues:
        for i in range(1, len(conv)):
            # Use up to max_history previous exchanges as context
            history = conv[max(0, i - max_history):i]
            reply = conv[i]  # The current reply
            inputs.append((history, reply))  # Store the context and the reply
    return inputs

# Build the dataset
data = build_inputs_targets(dialogues)

# Function to tokenize and encode the data using GPT-2 tokenizer
def encode_data(data, tokenizer):
    encoded_inputs = []
    for history, reply in data:
        input_text = "<|endoftext|>".join(history) + "<|endoftext|>"
        full_text = input_text + reply + "<|endoftext|>"
        encoded = tokenizer.encode(
            full_text,
            add_special_tokens=False,
            max_length=1024,
            truncation=True
        )
        encoded_inputs.append(torch.tensor(encoded))
    return encoded_inputs

# Encode the data
encoded_data = encode_data(data, tokenizer)

# Display sample encoded data
print("\nSample Encoded Data:")
for i in range(2):
    print(f"Encoded Sample {i+1}: {encoded_data[i][:10]}... (length: {len(encoded_data[i])})")


Sample Encoded Data:
Encoded Sample 1: tensor([ 6090,   356,   787,   428,  2068,    30,   220, 34821, 21952, 14769])... (length: 49)
Encoded Sample 2: tensor([ 6090,   356,   787,   428,  2068,    30,   220, 34821, 21952, 14769])... (length: 63)


- Prepare the data by creating input-target pairs, where the input is the conversation history and the target is the reply.
- The encode_data function tokenizes and encodes the combined text using the GPT-2 tokenizer.
- Ensure that the input sequences are truncated to a maximum length to fit the model's requirements.

### Create Dataset and DataLoader

In [5]:
# Custom dataset class for our chatbot data
class ChatDataset(Dataset):
    def __init__(self, encoded_data):
        self.data = encoded_data  # Store the encoded data

    def __len__(self):
        return len(self.data)  # Return the total number of samples

    def __getitem__(self, idx):
        return self.data[idx]  # Retrieve a sample by index

# Collate function to pad sequences within a batch and create attention masks and labels
def collate_fn(batch):
    """
    Collate function to process batches of data.
    Pads input sequences, creates attention masks and labels.
    """
    # Pad sequences to the maximum length in the batch
    input_ids = torch.nn.utils.rnn.pad_sequence(
        batch, batch_first=True, padding_value=tokenizer.pad_token_id
    )
    # Create attention masks (1 where input_ids is not pad_token_id, 0 otherwise)
    attention_mask = (input_ids != tokenizer.pad_token_id).long()
    # Create labels; set pad_token_id to -100 so that they are ignored in loss computation
    labels = input_ids.clone()
    labels[input_ids == tokenizer.pad_token_id] = -100
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

# To use a smaller subset of the data for faster training
subset_size = 30000  # Adjust based on our computational resources
encoded_data = encoded_data[:subset_size]

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

# First, split the data into training and test sets (80% train, 20% test)
train_data, test_data = train_test_split(encoded_data, test_size=0.2, random_state=42)

# Then, split the training data into training and validation sets (80% train, 20% val)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Verify the sizes of the splits
print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")

# Create dataset instances
train_dataset = ChatDataset(train_data)
val_dataset = ChatDataset(val_data)
test_dataset = ChatDataset(test_data)

# Define batch size
batch_size = 4  # Adjust based on available resources

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

Training set size: 19200
Validation set size: 4800
Test set size: 6000


- Create a custom ChatDataset class to handle our data.
- The collate_fn function pads sequences in a batch to the same length.
- Use DataLoader to efficiently load data during training.
- A subset of the data is used to speed up training when resources are limited

## Model Design and Initialization

In [6]:

# Specify the model name; use a smaller model if resources are limited
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Initialize the GPT-2 model
model_name = 'gpt2-medium'  # We can use 'gpt2-medium' if using a GPU

model = GPT2LMHeadModel.from_pretrained(model_name)
model.to(device)

# Display model size
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {count_parameters(model):,}")

Total Parameters: 354,823,168


Initialize the pre-trained GPT-2 model and move it to the appropriate device.

Load the GPT-2 model (chose 'gpt2-medium') and set up the device (CPU).

### Define Evaluation Metrics and Implement Evaluation Metrics

In [7]:
# Fine-Tuning the Model
epochs = 3  # Number of training epochs
optimizer = AdamW(model.parameters(), lr=3e-5)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=total_steps)


# Function to evaluate the model on a validation set
def evaluate(model, dataloader):
    """
    Evaluate the model on the validation set.
    Computes perplexity as the evaluation metric.
    """
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_tokens = 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            batch = {k: v.to(device) for k, v in batch.items()}
            # Forward pass with labels to compute loss
            outputs = model(**batch)
            loss = outputs.loss  # Mean loss per token in batch
            num_tokens = batch['attention_mask'].sum().item()
            # Accumulate loss and token counts
            total_loss += loss.item() * num_tokens  # Total loss over all tokens in batch
            total_tokens += num_tokens
    # Calculate perplexity
    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    return perplexity


# Function to calculate the BLEU score on the validation set
def calculate_bleu(model, dataloader, tokenizer):
    """
    Calculate the average BLEU score on the validation set.
    """
    model.eval()  # Set the model to evaluation mode
    bleu_scores = []
    smooth_fn = SmoothingFunction().method1  # Smoothing function for BLEU score
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating BLEU"):
            batch = {k: v.to(device) for k, v in batch.items()}
            # Generate predictions
            outputs = model.generate(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                max_length=batch['input_ids'].shape[1] + 50,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
            )
            for i in range(batch['input_ids'].size(0)):
                # Decode the reference and hypothesis texts
                input_len = (batch['input_ids'][i] != tokenizer.pad_token_id).sum().item()
                reference_ids = batch['input_ids'][i][input_len:].tolist()
                hypothesis_ids = outputs[i][input_len:].tolist()
                reference = tokenizer.decode(reference_ids, skip_special_tokens=True)
                hypothesis = tokenizer.decode(hypothesis_ids, skip_special_tokens=True)
                # Tokenize the texts
                reference_tokens = nltk.word_tokenize(reference)
                hypothesis_tokens = nltk.word_tokenize(hypothesis)
                # Compute BLEU score
                bleu = sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smooth_fn)
                bleu_scores.append(bleu)
    # Calculate average BLEU score
    average_bleu = sum(bleu_scores) / len(bleu_scores)
    return average_bleu

- The evaluate function computes the perplexity of the model on the validation set.
- Perplexity is a measure of how well the model predicts the sample; lower values indicate better performance.
- Use torch.no_grad() to disable gradient computation during evaluation for efficiency.

- The calculate_bleu function evaluates the model's predictions against the ground truth using the BLEU score.
- BLEU score measures the similarity between the generated text and reference text.
- Smoothing is applied to handle cases where the hypothesis and reference have few overlapping n-grams.

### Training Loop 

In [15]:
# Initialize variables for tracking best performance
best_val_perplexity = float('inf')
best_model_state = None

# Training loop
for epoch in range(epochs):
    total_train_loss = 0
    total_train_tokens = 0  # Initialize total_train_tokens
    model.train()  # Set the model to training mode
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()  # Reset gradients
        # Forward pass with labels to compute loss
        outputs = model(**batch)
        loss = outputs.loss  # Mean loss per token
        num_tokens = batch['attention_mask'].sum().item()
        # Backward pass to compute gradients
        loss.backward()
        # Update parameters
        optimizer.step()
        # Update learning rate
        scheduler.step()
        # Accumulate training loss
        total_train_loss += loss.item() * num_tokens
        total_train_tokens += num_tokens
    # Calculate average training loss
    avg_train_loss = total_train_loss / total_train_tokens
    print(f"Epoch {epoch+1}, Average Training Loss: {avg_train_loss:.4f}")
    # Evaluate on validation set
    val_perplexity = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}, Validation Perplexity: {val_perplexity:.4f}")
    if val_perplexity < best_val_perplexity:
        best_val_perplexity = val_perplexity
        best_model_state = model.state_dict()
        print(f"Validation perplexity improved; model saved.")
    else:
        print(f"No improvement in validation perplexity.")

# Load the best model state before testing
if best_model_state is not None:
    model.load_state_dict(best_model_state)
else:
    print("No improvement during training; using last epoch model.")

# Save the best model
model_save_path = 'fine_tuned_gpt2_medium.pt'
torch.save(model.state_dict(), model_save_path)
print(f"Best model saved to {model_save_path}")


Training Epoch 1:   0%|                                                                       | 0/4800 [00:01<?, ?it/s]


KeyboardInterrupt: 

- Train the model using the AdamW optimizer and a linear learning rate scheduler.
- The training loop iterates over epochs and batches, computing loss, performing backpropagation, and updating model parameters.
- After each epoch, evaluate the model on the validation set and compute perplexity.
- The trained model is saved to a file to avoid retraining in the future.

### Load the trained model

In [18]:
# Load the trained model
model_save_path = 'fine_tuned_gpt2_medium.pt'  # Define the model path here

try:
    model.load_state_dict(torch.load(model_save_path, map_location=device))
    print("Model loaded successfully!")
except RuntimeError as e:
    print(f"Error loading model state_dict: {e}")
    print("Please ensure that the model architecture matches the saved model weights.")

# Evaluate the final model on the test set
test_perplexity = evaluate(model, test_loader)
print(f"Final Test Perplexity: {test_perplexity:.4f}")

# Calculate BLEU score on the validation set
bleu_score = calculate_bleu(model, val_loader, tokenizer)
print(f"Validation BLEU Score: {bleu_score:.4f}")


C:\Users\Pete P\AppData\Local\Temp\ipykernel_12964\1028063248.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path, map_locat

Model loaded successfully!


Evaluating: 100%|████████████████████████████████████████████████████████████████| 1500/1500 [3:21:38<00:00,  8.07s/it]


Final Test Perplexity: 11.5385


Calculating BLEU: 100%|██████████████████████████████████████████████████████████| 1200/1200 [8:31:06<00:00, 25.56s/it]

Validation BLEU Score: 0.0122


- Load the saved model weights into the model architecture.
- The model is set to evaluation mode to disable dropout and other training-specific layers.
- This allows us to use the trained model for generating responses.

## Interaction with the Chatbot

In [19]:
# Implement Content Filtering
import re

# List of disallowed words or phrases to filter out
disallowed_words = [
    # Offensive Language
    r'self_harm', r'suicide', r'death', r'kill',
    # Profanity (replace with actual words or leave as *)
    #r'\b(f**k|s**t|b***h|d**n|a**hole|c**t|sl*t|pr*ck|b**tard)\b',
    # Slurs or Hate Speech 
    #r'\b(racial_slur1|racial_slur2|homophobic_slur1|homophobic_slur2|gender_slur|ethnic_slur|religious_slur)\b',  
    # Explicit Sexual Content
    r'\b(porn|nude|sex|explicit|xxx|erotic|fetish|incest|rape|molest)\b',
    # Spam or Malicious Content
    #r'\b(spam|phishing|malware|virus|adware|trojan|scam|fraud)\b',
    # Violence or Graphic Content
    r'\b(assault|gun|shooting|bomb|explosion|attack|blood|mutilation|massacre)\b',
    # Misinformation and Conspiracy Theories
    #r'\b(fake_news|hoax|conspiracy|anti_vax|flat_earth|election_fraud)\b',
    # Harmful Behavior or Self-Harm
    r'\b(cutting|eating_disorder|anorexia|bulimia|starving|addiction)\b',
]

# Escape special regex characters and compile the pattern
pattern = re.compile(r'\b(' + '|'.join([re.escape(word) for word in disallowed_words]) + r')\b', re.IGNORECASE)

def contains_disallowed_content(text):
    return bool(pattern.search(text))

- Define patterns for disallowed content to prevent the chatbot from generating inappropriate responses.
- The contains_disallowed_content function checks if the generated text contains any disallowed words or phrases.

### Chatbot Class Implementation

In [20]:
class Chatbot:
    def __init__(self, model, tokenizer, max_history=4):
        self.model = model  # The language generation model
        self.tokenizer = tokenizer  # Tokenizer to encode/decode text
        self.max_history = max_history  # Number of previous exchanges to consider
        self.chat_history = []  # Stores the conversation history
        self.device = device  # Ensure device is set
        # Set pad_token to eos_token for models like GPT-2 that do not have a pad_token
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def reset_history(self):
        self.chat_history = []

    def get_response(self, user_input):
        # Detect inappropriate or harmful content
        if contains_disallowed_content(user_input):
            return "I'm sorry, but I cannot assist with that request."
            
        self.chat_history.append(user_input)
        # Keep only the last max_history exchanges
        history = self.chat_history[-self.max_history:]
        # Combine the history into a single input string
        input_text = "<|endoftext|>".join(history) + "<|endoftext|>"
        
        # Encode the input text with truncation and padding
        encoding = self.tokenizer(
            input_text,
            return_tensors='pt',
            max_length=512,  # Truncate if necessary
            truncation=True,
            padding=True
        )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        # Generate a response
        output = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50,  # Generate up to 50 new tokens
            pad_token_id=self.tokenizer.pad_token_id,
            do_sample=True,  # Use sampling for more varied responses
            top_p=0.92,      # Nucleus sampling parameter
            top_k=40,        # Top-k sampling parameter
            temperature=0.8, # Sampling temperature
            eos_token_id=self.tokenizer.eos_token_id,
            num_return_sequences=1,  # Generate one response at a time
            repetition_penalty=1.3,  # Penalize repetition for better response diversity
            no_repeat_ngram_size=3   # Prevent repetition of n-grams
        )
        # Extract the generated tokens beyond the input length
        generated_tokens = output[0][input_ids.size(-1):]
        # Decode the generated tokens to text
        reply = self.tokenizer.decode(generated_tokens, skip_special_tokens=True)
        # Apply content filtering
        if contains_disallowed_content(reply):
            reply = "I'm sorry, but I can't assist with that request."
            
        self.chat_history.append(reply)
        return reply.strip()

- The Chatbot class encapsulates the chatbot's functionality.
- It maintains a conversation history to provide context for multi-turn conversations.
- The get_response method generates a response using the model and applies content filtering.
- Sampling parameters (top_p, top_k, temperature) control the diversity of the generated responses.

### Interact with the Chatbot via Console

In [21]:
# Instantiate the chatbot
chatbot = Chatbot(model, tokenizer)

# Start the conversation loop
print("Bot: Hello! How can I assist you today?")
while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit", "quit", "goodbye"]:
        print("Bot: Goodbye! Have a great day.")
        break
    response = chatbot.get_response(user_input)
    print(f"Bot: {response}")

Bot: Hello! How can I assist you today?


You:  Who are you?


Bot: A man who is a major player in this venture. He wants to keep his deal quiet, so I'll tell him nothing about it until after we've made some money... and then we can start talking again!  We have the best lawyer on


You:  Tell me a joke


Bot: What's your name? Tell us that now! What's my real last names like anyway!? Tell Us Your Name Now! What're You Calling Me? Answer Our Questions Here!! Please answer honestly without using profanity or insulting words!!!  If you


...

Bot: If you're thinking of giving up on life altogether... well... it's not that far from what my wife did years ago. She was going into business herself! She just couldn 't seem ta stay away for too long. When she retired her


You:  What do you think about AI or bias regarding it?


Bot: That depends... On the one hand if a machine is programmed to perceive reality then there must exist conscious agents capable both OF and WITHOUT its perception. For instance, consider our own sensory apparatus: eyes which project light rays at fixed distances through various pigments


You:  quit


Bot: Goodbye! Have a great day.


- Create an instance of the Chatbot class.
- A loop allows the user to interact with the chatbot via the console.
- The conversation continues until the user types an exit command.

In [1]:
!python app.py

^C
